## Generating point cloud

In [1]:
from pyodm import Node

In [2]:
n = Node('localhost', 3000)

In [3]:
from os import listdir

In [5]:
prefix = '../images/original'
files = [prefix + '/' + f for f in listdir(prefix)]

In [7]:
task = n.create_task(files, {'dsm': True})

In [8]:
task.wait_for_completion()

In [9]:
task.download_assets("/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm")

'/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm'

## Visualizind point cloud

In [8]:
import laspy
import open3d as o3d
import numpy as np
from os import listdir
from morph_texture import normalize

In [2]:
pc_data_dir = '/media/antonio/Ubuntail/Dropbox/education/phd/courses/processamento_imagens/assignments/final/pyodm/entwine_pointcloud/ept-data/'

In [5]:
points = None
for fname in listdir(pc_data_dir):
    pc = laspy.read(pc_data_dir + fname)
    point_data = np.stack([pc.X, pc.Y, pc.Z], axis=0).transpose((1, 0))
    if points is None:
        points = point_data
    else:
        points = np.concatenate([points, point_data])
points -= np.min(points, axis=0)
        

In [7]:
points = points.astype(np.float32)


dtype('int32')

In [9]:
surface = o3d.geometry.PointCloud()
surface.points = o3d.utility.Vector3dVector(points)
o3d.visualization.draw_geometries([surface])


In [27]:
import matplotlib.pyplot as plt

In [28]:
def show(img, plt_size=(16, 10), cmap=None):
    fig = plt.figure(figsize=plt_size)
    plt.imshow(img, cmap=cmap if cmap is not None else (None if len(img.shape) == 3 else 'gray'))

In [32]:
img = surface.Image

In [34]:
img.RGBDImage

<Type.RGBDImage: 9>

In [36]:
import cv2

In [72]:
len(points)

10771891

In [20]:
def points_to_image(points, resolution):
    H, W = resolution
    points = points.copy()
    points[:, 0] = normalize(points[:, 0], (0, W), points.dtype)
    points[:, 1] = normalize(points[:, 1], (0, H), points.dtype)
    points[:, 2] = normalize(points[:, 2], type=points.dtype)
    
    points[:, 0] = np.floor(points[:, 0]).astype(np.uint8)
    points[:, 1] = np.floor(points[:, 1]).astype(np.uint8)
    
    result = np.zeros(resolution, np.float128)
    for point in points:
        result[point[1], point[0]] += point[2]
    return result
    
    
    

In [17]:
img = np.zeros((500, 625))

In [21]:
dm = points_to_image(points, (500, 625))

In [22]:
dm.shape

(500, 625)

In [71]:
points[5449935]

array([146262,      0,      0], dtype=int32)

In [74]:
img = points_to_image(points[:, 0], points[:, 1], points[:, 2], (500, 625))

ValueError: invalid entry in coordinates array